# MovieLens 32M 数据探索性分析 (EDA)
本笔记本旨在通过可视化和统计分析，深入了解 MovieLens 数据集的特征，为后续召回和排序算法的设计提供依据。

## 准备工作：导入库与加载数据
在这个单元格中，我们将加载之前预处理好的 Parquet 文件。Parquet 格式相比 CSV 能显著提升数据读取速度。

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from pathlib import Path

# 设置绘图风格
plt.style.use('ggplot')
%matplotlib inline

# 加载处理后的数据
data_dir = Path('../data/processed')
ratings = pd.read_parquet(data_dir / 'ratings.parquet')
movies = pd.read_parquet(data_dir / 'movies.parquet')

print(f'成功加载 {len(ratings):,} 条评分数据和 {len(movies):,} 条电影元数据。')

## 1. 稀疏度分析 (Sparsity Analysis)
推荐系统的核心挑战之一是数据极其稀疏。在这个单元格中，我们计算用户-物品矩阵的填充比例。
- **稀疏度** = (1 - 实际评分数 / 理论最大评分数) * 100%
- 稀疏度越高，意味着我们对用户的偏好了解越少，推荐难度越大。

In [ ]:
n_users = ratings['userId'].nunique()
n_movies = ratings['movieId'].nunique()
actual_ratings = len(ratings)
possible_ratings = n_users * n_movies
sparsity = (1 - actual_ratings / possible_ratings) * 100

print(f'用户总数: {n_users:,}')
print(f'电影总数: {n_movies:,}')
print(f'理论最大评分组合: {possible_ratings:,}')
print(f'实际评分数量: {actual_ratings:,}')
print(f'数据集稀疏度: {sparsity:.4f}%')

## 2. 物品长尾效应分析 (Long-tail Effect)
互联网数据通常遵循幂律分布（长尾效应）。
- 我们将观察极少数的热门电影是否占据了绝大部分的评分。
- 如果长尾效应极强，热门召回（Popularity-based Recall）将是一个非常强力的基准模型。

In [ ]:
movie_counts = ratings.groupby('movieId').size().sort_values(ascending=False)

plt.figure(figsize=(12, 6))
plt.plot(movie_counts.values)
plt.title('电影流行度分布 (长尾效应)')
plt.xlabel('电影排名 (按流行度排序)')
plt.ylabel('评分次数')
plt.yscale('log')
plt.show()

top_10_percent_cutoff = int(len(movie_counts) * 0.1)
top_10_percent_ratings = movie_counts.iloc[:top_10_percent_cutoff].sum()
percentage = (top_10_percent_ratings / len(ratings)) * 100
print(f'前 10% 的热门电影占据了全站 {percentage:.2f}% 的评分。')

## 3. 用户活跃度分析
不同用户的观影量差异巨大。了解活跃度分布有助于我们判断：
- 是否需要为低频用户设计特殊的冷启动策略。
- 是否需要对“观影狂魔”进行降权，防止他们主导模型。

In [ ]:
user_counts = ratings.groupby('userId').size().sort_values(ascending=False)

plt.figure(figsize=(10, 5))
sns.histplot(user_counts, bins=50, kde=True)
plt.title('用户活跃度分布')
plt.xlabel('每个用户的评分数量')
plt.xscale('log')
plt.show()

print(f'用户平均评分数: {user_counts.mean():.1f}')
print(f'用户评分数中位数: {user_counts.median():.1f}')
print(f'最少评分数: {user_counts.min()}')
print(f'最大评分数: {user_counts.max()}')

## 4. 电影题材分布分析
了解数据集中的题材偏好，有助于我们设计基于内容的推荐（Content-based Recommendation）。

In [ ]:
genres_exploded = movies.explode('genres')
genre_counts = genres_exploded['genres'].value_counts()

plt.figure(figsize=(12, 6))
sns.barplot(x=genre_counts.values, y=genre_counts.index, palette='magma')
plt.title('电影题材分布')
plt.xlabel('电影数量')
plt.show()